<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>CU25_Modelo de gestión de Lista de Espera Quirúrgica</font></center>

<font color='steelblue'><b>Citizenlab Data Science Methodology > III - Feature Engineering Domain </font>
***
> # <font color='steelblue'> <b>15.- Feature Importance</font>

<font color='steelblue'>Feature Importance is the process that assigns scores to the input characteristics to a model, which indicate the relative importance of each characteristic, in order, for example, to be able to select the most important ones.</font>


## <font color='steelblue'>Tasks</font>

<font color='steelblue'><b>Perform Feature importance from model coefficients</b>			
- Linear Regression Feature importance		
- Logistic Regression Feature importance		
    
<font color='steelblue'><b>Perform Feature importance from Decision Tree			
- CART Feature Importance		
- Random Forest Regression Feature Importance		
    
<font color='steelblue'><b>Perform Feature importance from Permutation testing</b>			
    
<font color='steelblue'><b>Evaluate a Logistic Regression model with feature selection</b>			

## Consideraciones casos CitizenLab programados en R

* Algunas de las tareas de este proceso se han realizado en los notebooks del proceso 05 Data Collection porque eran necesarias para las tareas ETL. En esos casos, en este notebook se referencia al notebook del proceso 05 correspondiente
* Otras tareas típicas de este proceso se realizan en los notebooks del dominio IV al ser más eficiente realizarlas en el propio pipeline de modelización.
* Por tanto en los notebooks de este proceso de manera general se incluyen las comprobaciones necesarias, y comentarios si procede
* Las tareas del proceso se van a aplicar solo a los archivos que forman parte del despliegue, ya que hay muchos archivos intermedios que no procede pasar por este proceso
* El nombre de archivo del notebook hace referencia al nombre de archivo del proceso 05 al que se aplica este proceso, por eso pueden no ser correlativa la numeración
* Las comprobaciones se van a realizar teniendo en cuenta que el lenguaje utilizado en el despliegue de este caso es R

## <font color='green'>File</font>

- <font color='steelblue'> <b>Input File: CU_25_09.2_01_lista_espera_completo_clean_v_01.csv</font>
- <font color='steelblue'> <b>Output File: No aplica</font>

### Encoding

Con la siguiente expresión se evitan problemas con el encoding al ejecutar el notebook. Es posible que deba ser eliminada o adaptada a la máquina en la que se ejecute el código.

In [18]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "LC_COLLATE=es_ES.UTF-8;LC_CTYPE=es_ES.UTF-8;LC_MONETARY=es_ES.UTF-8;LC_NUMERIC=C;LC_TIME=es_ES.UTF-8"

## <font color='green'>Settings</font>

### Libraries to use

In [19]:
library(readr)
library(dplyr)
library(tidyr)
library(forcats)
library(lubridate)
library(rpart)

### Paths

In [20]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if using this option

In [21]:
# file_data <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [22]:
iFile <- "CU_25_09.2_01_lista_espera_completo_clean_v_01.csv"
file_data <- paste0(iPath, iFile)

if(file.exists(file_data)){
    cat("Se leerán datos del archivo: ", file_data)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/CU_25_09.2_01_lista_espera_completo_clean_v_01.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [23]:
data <- read.csv(file_data)


Estructura de  los datos:

In [24]:
data |> glimpse()

Rows: 55,216
Columns: 46
$ Hospital          <chr> "HOSPITAL REY JUAN CARLOS", "HOSPITAL CENTRAL DE LA …
$ Especialidad      <chr> "UROLOGÍA", "ODONTOESTOMATOLOGÍA", "GINECOLOGÍA", "D…
$ total_pacientes   <int> 344, 0, 52, 37, 0, 4, 0, 718, 0, 271, 108, 0, 34, 86…
$ ano               <int> 2021, 2020, 2021, 2021, 2021, 2020, 2021, 2020, 2021…
$ semana            <int> 30, 36, 49, 23, 3, 5, 50, 7, 35, 1, 42, 10, 21, 33, …
$ CODCNH            <int> 281348, 280724, 281292, 281292, 281236, 280724, 2807…
$ id_area           <int> 8, 7, 11, 11, 11, 7, 3, 6, 1, 2, 2, 8, 11, 11, 1, 3,…
$ nombre_area       <chr> "SUR-OESTE I", "CENTRO-OESTE", "SUR II", "SUR II", "…
$ cmunicipio        <int> 280920, 280796, 280133, 280133, 281610, 280796, 2800…
$ Municipio         <chr> "MÓSTOLES", "MADRID", "ARANJUEZ", "ARANJUEZ", "VALDE…
$ CAMAS             <int> 382, 475, 98, 98, 182, 475, 507, 613, 269, 1143, 156…
$ Clase             <chr> "HOSPITALES GENERALES", "HOSPITALES GENERALES", "HOS…
$ Dependencia  

Muestra de los primeros datos:

In [25]:
data |> slice_head(n = 5)

Hospital,Especialidad,total_pacientes,ano,semana,CODCNH,id_area,nombre_area,cmunicipio,Municipio,⋯,t9_1,t10_1,t11_1,t12_1,capacidad,pacientes,consultas,hospitalizaciones,Target,is_train
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<lgl>
HOSPITAL REY JUAN CARLOS,UROLOGÍA,344,2021,30,281348,8,SUR-OESTE I,280920,MÓSTOLES,⋯,0.3348780,0.1369254,0.5072726,0.5849309,17,1447,573,12,54.45,TRUE
HOSPITAL CENTRAL DE LA DEFENSA GOMEZ ULLA,ODONTOESTOMATOLOGÍA,0,2020,36,280724,7,CENTRO-OESTE,280796,MADRID,⋯,0.4914365,0.1217100,0.4915713,0.5597213,0,1211,45,0,0.00,TRUE
HOSPITAL UNIVERSITARIO DEL TAJO,GINECOLOGÍA,52,2021,49,281292,11,SUR II,280133,ARANJUEZ,⋯,0.3346063,0.1517321,0.5024130,0.5900028,8,1293,108,2,37.96,TRUE
HOSPITAL UNIVERSITARIO DEL TAJO,DERMATOLOGÍA,37,2021,23,281292,11,SUR II,280133,ARANJUEZ,⋯,0.3346063,0.1517321,0.5024130,0.5900028,5,1501,103,2,23.14,TRUE
HOSPITAL UNIVERSITARIO INFANTA ELENA,ODONTOESTOMATOLOGÍA,0,2021,3,281236,11,SUR II,281610,VALDEMORO,⋯,0.3346063,0.1517321,0.5024130,0.5900028,0,1240,44,0,0.00,TRUE


## Feature importance from model coefficients

### Linear Regression Feature importance

In [26]:
# Fit a linear regression model
model <- lm(Target ~ ., data = data)

# Extract the coefficients and their names
coefficients <- coef(model)
names <- names(coefficients)

# Calculate the absolute values of the coefficients
abs_coefficients <- abs(coefficients)

# Sort the coefficients in descending order
sorted_coefficients <- sort(abs_coefficients, decreasing = TRUE)

# Print the feature importance
feature_importance <- data.frame(Feature = names(sorted_coefficients), Importance = sorted_coefficients)
print(feature_importance)

                                                                                                                Feature
(Intercept)                                                                                                 (Intercept)
HospitalHOSPITAL GENERAL DE VILLALBA                                               HospitalHOSPITAL GENERAL DE VILLALBA
HospitalHOSPITAL REY JUAN CARLOS                                                       HospitalHOSPITAL REY JUAN CARLOS
HospitalHOSPITAL UNIVERSITARIO DE TORREJON                                   HospitalHOSPITAL UNIVERSITARIO DE TORREJON
HospitalHOSPITAL UNIVERSITARIO PUERTA DE HIERRO MAJADAHONDA HospitalHOSPITAL UNIVERSITARIO PUERTA DE HIERRO MAJADAHONDA
HospitalHOSPITAL UNIVERSITARIO INFANTA CRISTINA                         HospitalHOSPITAL UNIVERSITARIO INFANTA CRISTINA
HospitalHOSPITAL UNIVERSITARIO DEL TAJO                                         HospitalHOSPITAL UNIVERSITARIO DEL TAJO
HospitalHOSPITAL UNIVERSITARIO DEL SURES

### Logistic Regression Feature importance

No aplica

## Decision Tree Feature Importance

### CART Regression Feature Importance

In [27]:
# Fit a CART regression model
model <- rpart(Target ~ ., data = data, method = "anova")

# Calculate feature importance
importance <- round(model$variable.importance, 2)

# Sort the feature importance in descending order
sorted_importance <- sort(importance, decreasing = TRUE)

# Print the feature importance
feature_importance <- data.frame(Feature = names(sorted_importance), Importance = sorted_importance)
print(feature_importance)

                            Feature  Importance
total_pacientes     total_pacientes 46585351.57
consultas                 consultas 42984648.71
hospitalizaciones hospitalizaciones 36939615.15
Hospital                   Hospital 24889013.04
Especialidad           Especialidad 24414140.39
capacidad                 capacidad 23312800.43
DIAL                           DIAL 11881062.31
Municipio                 Municipio 10285219.66
TAC                             TAC  9819026.12
CAMAS                         CAMAS  8508724.45
ASD                             ASD  6396715.74
HEM                             HEM  2251712.22
DO                               DO  1863984.85
RM                               RM  1700792.14
nombre_area             nombre_area  1546078.05
Y                                 Y  1244093.28
id_area                     id_area  1122111.35
CODCNH                       CODCNH   804724.89
t12_1                         t12_1   598653.71
X                                 X   49

### CART Classification Feature Importance

In [28]:
# # Fit a CART classification model
# library(rpart)
# model <- rpart(Target ~ ., data = data, method = "class")

# # Calculate feature importance
# importance <- round(model$variable.importance, 2)

# # Sort the feature importance in descending order
# sorted_importance <- sort(importance, decreasing = TRUE)

# # Print the feature importance
# feature_importance <- data.frame(Feature = names(sorted_importance), Importance = sorted_importance)
# print(feature_importance)

### Random Forest Regression Feature Importance

In [29]:
# # Fit a Random Forest regression model
# library(randomForest)
# model <- randomForest(Target ~ ., data = data)

# # Get feature importance
# importance <- importance(model)

# # Print the feature importance
# print(importance)

### Random Forest Classification Feature Importance

No aplica

## Permutation Feature Importance

### Permutation Feature Importance for Regression

In [30]:
# # Install and load the 'iml' package

# # Fit a regression model
# model <- lm(Target ~ ., data = data)

# # Create an instance of the 'FeatureImp' class
# feature_imp <- FeatureImp$new(model, data = data, y = data$Target)

# # Calculate permutation feature importance
# perm_importance <- feature_imp$permutation()

# # Print the permutation feature importance
# print(perm_importance)

### Permutation Feature Importance for Classification

No aplica

## Evaluating a Regression model with feature selection.

### Evaluating with all selected features.

In [56]:

library(rpart)
library(dplyr)
library(caTools)

# Split the data into training and testing sets based on the "is_train" column
train_data <- filter(data, is_train == 1)
test_data <- filter(data, is_train == 0)


# Select the top 5 most important features
selected_features <- colnames(data)


train_data_selected <- select(train_data, all_of(selected_features), Target)

# Train a Linear Regression model with the selected features
model_lm <- lm(Target ~ ., data = train_data_selected)

# Select the same features from the test data
test_data_selected <- select(test_data, all_of(selected_features), Target)
# Make predictions on the test data
predictions <- predict(model_lm, newdata = test_data_selected)

# Calculate Mean Squared Error (MSE)
mse <- mean((predictions - test_data_selected$Target)^2)
print(paste("Mean Squared Error:", round(mse, 4)))

[1] "Mean Squared Error: 917.7152"


### Evaluating with feature selection performed using feature importance.

<font color='tomato'>Select type of Feature Importance to use</font>

Las 5 variables con mayor importancia

<font color='darkgrey'><b>Operation</font>

In [62]:
colnames(data)

[1] "Hospital"          "Especialidad"      "total_pacientes"  
 [4] "ano"               "semana"            "CODCNH"           
 [7] "id_area"           "nombre_area"       "cmunicipio"       
[10] "Municipio"         "CAMAS"             "Clase"            
[13] "Dependencia"       "TAC"               "RM"               
[16] "GAM"               "HEM"               "ASD"              
[19] "ALI"               "SPECT"             "MAMOS"            
[22] "DO"                "DIAL"              "X"                
[25] "Y"                 "t3_1"              "t1_1"             
[28] "t2_1"              "t2_2"              "t4_1"             
[31] "t4_2"              "t4_3"              "t5_1"             
[34] "t6_1"              "t7_1"              "t8_1"             
[37] "t9_1"              "t10_1"             "t11_1"            
[40] "t12_1"             "capacidad"         "pacientes"        
[43] "consultas"         "hospitalizaciones" "Target"           
[46] "is_train"

In [66]:

library(rpart)
library(dplyr)
library(caTools)

# Split the data into training and testing sets based on the "is_train" column
train_data <- filter(data, is_train == 1)
test_data <- filter(data, is_train == 0)

# Train a CART regression model to calculate feature importance
model_cart <- rpart(Target ~ ., data = train_data, method = "anova")

# Get the feature importance
importance <- round(model_cart$variable.importance, 2)

# Select the top 5 most important features
selected_features <- names(importance)[order(importance, decreasing = TRUE)][1:18]
selected_features

train_data_selected <- select(train_data, all_of(selected_features), Target)

# Train a Linear Regression model with the selected features
model_lm <- lm(Target ~ ., data = train_data_selected)

# Select the same features from the test data
test_data_selected <- select(test_data, all_of(selected_features), Target)
# Make predictions on the test data
predictions <- predict(model_lm, newdata = test_data_selected)

# Calculate Mean Squared Error (MSE)
mse <- mean((predictions - test_data_selected$Target)^2)
print(paste("Mean Squared Error:", round(mse, 4)))



[1] "total_pacientes"   "consultas"         "hospitalizaciones"
 [4] "Hospital"          "Especialidad"      "capacidad"        
 [7] "DIAL"              "Municipio"         "TAC"              
[10] "CAMAS"             "ASD"               "HEM"              
[13] "DO"                "RM"                "nombre_area"      
[16] "Y"                 "id_area"           "CODCNH"

[1] "Mean Squared Error: 923.2858"


## <font color='green'>Data Save</font>

* No aplica

<font color='tomato'> Identificamos los datos a guardar</font>

In [ ]:
data_to_save <- data


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_06".
* Resto del nombre del archivo de entrada
* Extensión del archivo

Ejemplo: "CU_04_06_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas) y que se ha transformado en el proceso 06

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 15

In [ ]:
# caso <- "CU_XX"
# proceso <- '_09.2'
# tarea <- "_XX"
# archivo <- ""
# proper <- "_xxxxx"
# extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar datos por datos_xx si es necesario

In [ ]:
# file_save <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save_xxxxx, path_out)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [ ]:
# file_save <- paste0(caso, proceso, tarea, archivo, proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save_xxxxx, path_out)

# cat('File saved as: ')
# path_out

#### Copia del fichero a Input

Si el archivo se va a usar en otros notebooks, copiar a la carpeta Input

In [ ]:
# path_in <- paste0(iPath, file_save)
# file.copy(path_out, path_in, overwrite = TRUE)

## <font color='#2874a6'>REPORT</font>

A continuación se realizará un informe de las acciones realizadas

## <font color=' #2874a6 '>Main Actions Carried Out</font>

- Si eran necesarias se han realizado en el proceso 05 por cuestiones de eficiencia
- O bien se hacen en el dominio IV o V para integrar en el pipeline de modelización

## <font color=' #2874a6 '>Main Conclusions</font>

- Los datos están listos para la modelización y despliegue


## <font color='RED'>CODE TO DEPLOY (PILOT)</font>

A continuación se incluirá el código que deba ser llevado a despliegue para producción, dado que se entiende efectúa operaciones necesarias sobre los datos en la ejecución del prototipo

<font color=' red '><b>Description</b></font>

- No hay nada que desplegar en el piloto, ya que estos datos son estáticos o en todo caso cambian con muy poca frecuencia, altamente improbable durante el proyecto.


<font color=' red '><b>CODE</b></font>